In [103]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") 
      else "NOT AVAILABLE")

Version:  2.6.0-dev20210510
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [2]:
# Downloading the IMDB dataset
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [85]:
# Preparing train, test and validation datasets
batch_size = 32
seed = 42

train_data = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

validation_data = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

test_data = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
validation_data = validation_data.cache().prefetch(buffer_size=AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size=AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [86]:
# Exploring the data
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
print("Train example batch: \n")
print(train_examples_batch)
print('\n\n\n\nLabels:\n')
print(train_labels_batch)

Train example batch: 

tf.Tensor(
[[b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
  b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes a

  b'Not only was this movie better than all the final season of H:LOTS. But it was better than any movie made for TV I have ever seen!<br /><br />Looking at the "Top 250" I see that only one small screen movie has made it: How the Grinch Stole Christmas. I think it is time to increase that group to 2.<br /><br />I will admit that the original series had several shows that were better than this, but I didn\'t mind. I just LOVED being able to enter the world of the Baltimore Homicide Squad again!']], shape=(10, 32), dtype=string)




Labels:

tf.Tensor(
[[0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1]
 [1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 0]
 [0 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 1]
 [1 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1]
 [0 1 0 1 1 1 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 0 1]
 [0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0]
 [0 1 1 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 1 

In [87]:
train_examples_batch = train_examples_batch.numpy()[0]
train_labels_batch = train_labels_batch.numpy()[0]

validation_data_batch = next(iter(validation_data.batch(32))) 
train_data_batch = next(iter(train_data.batch(32)))


In [88]:
next(iter(validation_data.batch(32))) 

(<tf.Tensor: shape=(32, 32), dtype=string, numpy=
 array([[b"(spoilers)<br /><br />I shoulda figured. The dvd didn't even show the time or how long it was. I thought Wild Cardz was the worst anime movie I'd ever seen. This one is much worse. Makes no sense. Thrown together plot. All so we can see oversized breasts on a figure that doesn't support them. <br /><br />It had to of been a student film. That musta gotten a C grade<br /><br />2/10<br /><br />Quality: 1/10 Entertainment: 6/10 (until I found out it wasn't going to end.) the ending made no darn sense<br /><br />Replayable: 0/10",
         b"What this movie fails from answering is how wrong this war is (and most US wars recently made only to get some oil).<br /><br />How many innocent civilian casualties there has been, how many lives perished and how blatantly stupid the perpetrators are.<br /><br />So, let me ask you - if American soldier kills women and children apart from enemy, its OK, but if government accidentally kills th

In [89]:
# Building the model
# First embedding layer
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.964519  ,  0.12235046, -0.35730854,  0.48033378,  0.18034822,
        -0.18373251, -0.03941955, -0.22600542, -0.32746387,  0.3865721 ,
         0.2286384 ,  0.00236568, -0.07970965,  0.06908172, -0.273917  ,
        -0.25015512, -0.17116444,  0.10235601,  0.13415377, -0.6560994 ,
        -0.01646905, -0.18063518,  0.7175045 ,  0.15740581, -0.57366866,
         0.02249735, -1.2577188 ,  0.21991247,  0.5483787 , -0.2534604 ,
        -0.5942718 , -0.05421532,  0.2442439 , -0.02258026, -0.22258581,
         0.60274124, -0.02160914,  0.07075842,  0.09879221, -0.44222328,
        -0.02986133,  0.19645615, -0.61093885, -0.18569276, -0.23887902,
        -0.3891516 , -0.35830313, -0.25282302,  0.3660045 ,  0.06032449],
       [ 0.76805687,  0.37268633,  0.5389303 ,  0.38905075, -0.08719452,
        -0.40928397, -0.09052341, -0.35153955, -0.58467203, -0.08505731,
        -0.25557598,  0.06093939, -0.147848  ,  0.17880681,  0.25342456,
 

In [90]:
# Full model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_8 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_4 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [91]:
# compiling model
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [95]:
# Training the model
history = model.fit(train_data,
                   epochs=5,
                   validation_data=validation_data,
                   verbose=1)

Epoch 1/5


185/625 [=======>......................] - ETA: 10:52 - loss: 0.3887 - accuracy: 0.843 - ETA: 8:35 - loss: 0.3904 - accuracy: 0.859 - ETA: 8:35 - loss: 0.3682 - accuracy: 0.87 - ETA: 8:44 - loss: 0.3625 - accuracy: 0.89 - ETA: 8:41 - loss: 0.3594 - accuracy: 0.90 - ETA: 8:37 - loss: 0.3570 - accuracy: 0.89 - ETA: 8:35 - loss: 0.3473 - accuracy: 0.90 - ETA: 8:40 - loss: 0.3428 - accuracy: 0.90 - ETA: 8:46 - loss: 0.3413 - accuracy: 0.89 - ETA: 8:43 - loss: 0.3442 - accuracy: 0.89 - ETA: 8:39 - loss: 0.3446 - accuracy: 0.89 - ETA: 8:37 - loss: 0.3442 - accuracy: 0.89 - ETA: 8:34 - loss: 0.3387 - accuracy: 0.89 - ETA: 8:32 - loss: 0.3389 - accuracy: 0.90 - ETA: 8:32 - loss: 0.3338 - accuracy: 0.90 - ETA: 8:32 - loss: 0.3340 - accuracy: 0.90 - ETA: 8:32 - loss: 0.3408 - accuracy: 0.90 - ETA: 8:31 - loss: 0.3408 - accuracy: 0.89 - ETA: 8:30 - loss: 0.3398 - accuracy: 0.88 - ETA: 8:29 - loss: 0.3409 - accuracy: 0.88 - ETA: 8:27 - loss: 0.3381 - accuracy: 0.88 - ETA: 8:25 - loss: 0.3353 - acc

371/625 [================>.............] - ETA: 6:08 - loss: 0.3521 - accuracy: 0.84 - ETA: 6:07 - loss: 0.3520 - accuracy: 0.84 - ETA: 6:07 - loss: 0.3519 - accuracy: 0.84 - ETA: 6:06 - loss: 0.3520 - accuracy: 0.85 - ETA: 6:05 - loss: 0.3521 - accuracy: 0.85 - ETA: 6:04 - loss: 0.3524 - accuracy: 0.85 - ETA: 6:03 - loss: 0.3521 - accuracy: 0.85 - ETA: 6:02 - loss: 0.3528 - accuracy: 0.84 - ETA: 6:02 - loss: 0.3530 - accuracy: 0.84 - ETA: 6:01 - loss: 0.3531 - accuracy: 0.84 - ETA: 6:00 - loss: 0.3537 - accuracy: 0.84 - ETA: 5:59 - loss: 0.3536 - accuracy: 0.84 - ETA: 5:58 - loss: 0.3536 - accuracy: 0.84 - ETA: 5:57 - loss: 0.3530 - accuracy: 0.85 - ETA: 5:57 - loss: 0.3524 - accuracy: 0.85 - ETA: 5:56 - loss: 0.3528 - accuracy: 0.84 - ETA: 5:56 - loss: 0.3518 - accuracy: 0.85 - ETA: 5:55 - loss: 0.3517 - accuracy: 0.85 - ETA: 5:55 - loss: 0.3520 - accuracy: 0.85 - ETA: 5:54 - loss: 0.3522 - accuracy: 0.84 - ETA: 5:53 - loss: 0.3519 - accuracy: 0.84 - ETA: 5:53 - loss: 0.3529 - accura

557/625 [=========================>....] - ETA: 3:44 - loss: 0.3420 - accuracy: 0.85 - ETA: 3:44 - loss: 0.3419 - accuracy: 0.85 - ETA: 3:43 - loss: 0.3415 - accuracy: 0.85 - ETA: 3:42 - loss: 0.3420 - accuracy: 0.85 - ETA: 3:42 - loss: 0.3419 - accuracy: 0.85 - ETA: 3:41 - loss: 0.3418 - accuracy: 0.85 - ETA: 3:40 - loss: 0.3416 - accuracy: 0.85 - ETA: 3:40 - loss: 0.3417 - accuracy: 0.85 - ETA: 3:39 - loss: 0.3415 - accuracy: 0.85 - ETA: 3:38 - loss: 0.3417 - accuracy: 0.85 - ETA: 3:38 - loss: 0.3414 - accuracy: 0.85 - ETA: 3:37 - loss: 0.3411 - accuracy: 0.85 - ETA: 3:36 - loss: 0.3407 - accuracy: 0.85 - ETA: 3:36 - loss: 0.3409 - accuracy: 0.85 - ETA: 3:35 - loss: 0.3407 - accuracy: 0.85 - ETA: 3:34 - loss: 0.3407 - accuracy: 0.85 - ETA: 3:34 - loss: 0.3407 - accuracy: 0.85 - ETA: 3:33 - loss: 0.3404 - accuracy: 0.85 - ETA: 3:32 - loss: 0.3399 - accuracy: 0.85 - ETA: 3:32 - loss: 0.3394 - accuracy: 0.85 - ETA: 3:31 - loss: 0.3391 - accuracy: 0.85 - ETA: 3:30 - loss: 0.3389 - accura

625/625 [==============================] - ETA: 1:05 - loss: 0.3262 - accuracy: 0.85 - ETA: 1:04 - loss: 0.3260 - accuracy: 0.85 - ETA: 1:03 - loss: 0.3258 - accuracy: 0.85 - ETA: 1:02 - loss: 0.3258 - accuracy: 0.85 - ETA: 1:01 - loss: 0.3259 - accuracy: 0.85 - ETA: 1:00 - loss: 0.3258 - accuracy: 0.85 - ETA: 59s - loss: 0.3261 - accuracy: 0.8583 - ETA: 58s - loss: 0.3262 - accuracy: 0.858 - ETA: 57s - loss: 0.3261 - accuracy: 0.858 - ETA: 56s - loss: 0.3260 - accuracy: 0.858 - ETA: 55s - loss: 0.3261 - accuracy: 0.858 - ETA: 54s - loss: 0.3261 - accuracy: 0.858 - ETA: 53s - loss: 0.3263 - accuracy: 0.858 - ETA: 53s - loss: 0.3260 - accuracy: 0.858 - ETA: 52s - loss: 0.3259 - accuracy: 0.858 - ETA: 51s - loss: 0.3257 - accuracy: 0.858 - ETA: 50s - loss: 0.3259 - accuracy: 0.858 - ETA: 49s - loss: 0.3258 - accuracy: 0.858 - ETA: 48s - loss: 0.3259 - accuracy: 0.858 - ETA: 47s - loss: 0.3261 - accuracy: 0.858 - ETA: 46s - loss: 0.3263 - accuracy: 0.858 - ETA: 45s - loss: 0.3262 - accura

...

625/625 [==============================] - ETA: 53s - loss: 0.0227 - accuracy: 0.995 - ETA: 52s - loss: 0.0227 - accuracy: 0.995 - ETA: 51s - loss: 0.0227 - accuracy: 0.995 - ETA: 50s - loss: 0.0227 - accuracy: 0.995 - ETA: 50s - loss: 0.0227 - accuracy: 0.995 - ETA: 49s - loss: 0.0227 - accuracy: 0.995 - ETA: 48s - loss: 0.0228 - accuracy: 0.995 - ETA: 47s - loss: 0.0228 - accuracy: 0.995 - ETA: 46s - loss: 0.0228 - accuracy: 0.995 - ETA: 46s - loss: 0.0227 - accuracy: 0.995 - ETA: 45s - loss: 0.0227 - accuracy: 0.995 - ETA: 44s - loss: 0.0227 - accuracy: 0.995 - ETA: 43s - loss: 0.0227 - accuracy: 0.995 - ETA: 42s - loss: 0.0227 - accuracy: 0.995 - ETA: 42s - loss: 0.0226 - accuracy: 0.995 - ETA: 41s - loss: 0.0226 - accuracy: 0.995 - ETA: 40s - loss: 0.0226 - accuracy: 0.995 - ETA: 39s - loss: 0.0226 - accuracy: 0.995 - ETA: 38s - loss: 0.0230 - accuracy: 0.995 - ETA: 37s - loss: 0.0231 - accuracy: 0.995 - ETA: 37s - loss: 0.0230 - accuracy: 0.995 - ETA: 36s - loss: 0.0230 - accurac

186/625 [=======>......................] - ETA: 8:13 - loss: 0.0039 - accuracy: 1.00 - ETA: 8:24 - loss: 0.0100 - accuracy: 1.00 - ETA: 8:23 - loss: 0.0082 - accuracy: 1.00 - ETA: 8:24 - loss: 0.0075 - accuracy: 1.00 - ETA: 8:25 - loss: 0.0222 - accuracy: 0.99 - ETA: 8:22 - loss: 0.0191 - accuracy: 0.99 - ETA: 8:23 - loss: 0.0166 - accuracy: 0.99 - ETA: 8:22 - loss: 0.0155 - accuracy: 0.99 - ETA: 8:20 - loss: 0.0139 - accuracy: 0.99 - ETA: 8:20 - loss: 0.0264 - accuracy: 0.99 - ETA: 8:19 - loss: 0.0249 - accuracy: 0.99 - ETA: 8:19 - loss: 0.0232 - accuracy: 0.99 - ETA: 8:18 - loss: 0.0218 - accuracy: 0.99 - ETA: 8:20 - loss: 0.0207 - accuracy: 0.99 - ETA: 8:19 - loss: 0.0206 - accuracy: 0.99 - ETA: 8:18 - loss: 0.0230 - accuracy: 0.99 - ETA: 8:16 - loss: 0.0232 - accuracy: 0.99 - ETA: 8:14 - loss: 0.0221 - accuracy: 0.99 - ETA: 8:12 - loss: 0.0210 - accuracy: 0.99 - ETA: 8:12 - loss: 0.0204 - accuracy: 0.99 - ETA: 8:12 - loss: 0.0200 - accuracy: 0.99 - ETA: 8:10 - loss: 0.0194 - accura

372/625 [================>.............] - ETA: 6:01 - loss: 0.0104 - accuracy: 0.99 - ETA: 6:00 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:59 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:58 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:57 - loss: 0.0105 - accuracy: 0.99 - ETA: 5:57 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:56 - loss: 0.0105 - accuracy: 0.99 - ETA: 5:55 - loss: 0.0105 - accuracy: 0.99 - ETA: 5:54 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:53 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:52 - loss: 0.0105 - accuracy: 0.99 - ETA: 5:51 - loss: 0.0105 - accuracy: 0.99 - ETA: 5:50 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:49 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:48 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:47 - loss: 0.0104 - accuracy: 0.99 - ETA: 5:46 - loss: 0.0103 - accuracy: 0.99 - ETA: 5:45 - loss: 0.0103 - accuracy: 0.99 - ETA: 5:44 - loss: 0.0103 - accuracy: 0.99 - ETA: 5:43 - loss: 0.0103 - accuracy: 0.99 - ETA: 5:43 - loss: 0.0103 - accuracy: 0.99 - ETA: 5:42 - loss: 0.0103 - accura

558/625 [=========================>....] - ETA: 3:25 - loss: 0.0144 - accuracy: 0.99 - ETA: 3:25 - loss: 0.0144 - accuracy: 0.99 - ETA: 3:24 - loss: 0.0144 - accuracy: 0.99 - ETA: 3:23 - loss: 0.0145 - accuracy: 0.99 - ETA: 3:22 - loss: 0.0147 - accuracy: 0.99 - ETA: 3:21 - loss: 0.0150 - accuracy: 0.99 - ETA: 3:21 - loss: 0.0156 - accuracy: 0.99 - ETA: 3:20 - loss: 0.0159 - accuracy: 0.99 - ETA: 3:19 - loss: 0.0159 - accuracy: 0.99 - ETA: 3:18 - loss: 0.0159 - accuracy: 0.99 - ETA: 3:17 - loss: 0.0158 - accuracy: 0.99 - ETA: 3:17 - loss: 0.0158 - accuracy: 0.99 - ETA: 3:16 - loss: 0.0158 - accuracy: 0.99 - ETA: 3:15 - loss: 0.0158 - accuracy: 0.99 - ETA: 3:14 - loss: 0.0158 - accuracy: 0.99 - ETA: 3:14 - loss: 0.0159 - accuracy: 0.99 - ETA: 3:13 - loss: 0.0162 - accuracy: 0.99 - ETA: 3:12 - loss: 0.0164 - accuracy: 0.99 - ETA: 3:11 - loss: 0.0165 - accuracy: 0.99 - ETA: 3:10 - loss: 0.0167 - accuracy: 0.99 - ETA: 3:10 - loss: 0.0168 - accuracy: 0.99 - ETA: 3:09 - loss: 0.0172 - accura

625/625 [==============================] - ETA: 53s - loss: 0.0160 - accuracy: 0.996 - ETA: 53s - loss: 0.0160 - accuracy: 0.996 - ETA: 52s - loss: 0.0160 - accuracy: 0.996 - ETA: 51s - loss: 0.0160 - accuracy: 0.996 - ETA: 50s - loss: 0.0159 - accuracy: 0.996 - ETA: 49s - loss: 0.0159 - accuracy: 0.996 - ETA: 48s - loss: 0.0159 - accuracy: 0.996 - ETA: 48s - loss: 0.0159 - accuracy: 0.996 - ETA: 47s - loss: 0.0158 - accuracy: 0.996 - ETA: 46s - loss: 0.0158 - accuracy: 0.996 - ETA: 45s - loss: 0.0158 - accuracy: 0.996 - ETA: 44s - loss: 0.0158 - accuracy: 0.996 - ETA: 43s - loss: 0.0158 - accuracy: 0.996 - ETA: 43s - loss: 0.0157 - accuracy: 0.996 - ETA: 42s - loss: 0.0158 - accuracy: 0.996 - ETA: 41s - loss: 0.0158 - accuracy: 0.996 - ETA: 40s - loss: 0.0158 - accuracy: 0.996 - ETA: 39s - loss: 0.0157 - accuracy: 0.996 - ETA: 39s - loss: 0.0159 - accuracy: 0.995 - ETA: 38s - loss: 0.0159 - accuracy: 0.995 - ETA: 37s - loss: 0.0158 - accuracy: 0.996 - ETA: 36s - loss: 0.0158 - accurac

In [98]:
# Evaluating the model
loss, accuracy = model.evaluate(test_data)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

543/782 [===================>..........] - ETA: 2:51 - loss: 0.7991 - accuracy: 0.87 - ETA: 14s - loss: 0.3752 - accuracy: 0.9062 - ETA: 13s - loss: 0.4861 - accuracy: 0.875 - ETA: 13s - loss: 0.6179 - accuracy: 0.862 - ETA: 14s - loss: 0.7299 - accuracy: 0.838 - ETA: 13s - loss: 0.7506 - accuracy: 0.828 - ETA: 14s - loss: 0.6944 - accuracy: 0.835 - ETA: 14s - loss: 0.6886 - accuracy: 0.836 - ETA: 14s - loss: 0.6764 - accuracy: 0.838 - ETA: 14s - loss: 0.6813 - accuracy: 0.837 - ETA: 14s - loss: 0.6941 - accuracy: 0.838 - ETA: 14s - loss: 0.6921 - accuracy: 0.838 - ETA: 14s - loss: 0.6804 - accuracy: 0.839 - ETA: 14s - loss: 0.6455 - accuracy: 0.846 - ETA: 14s - loss: 0.6606 - accuracy: 0.843 - ETA: 15s - loss: 0.6776 - accuracy: 0.841 - ETA: 15s - loss: 0.6710 - accuracy: 0.841 - ETA: 15s - loss: 0.6761 - accuracy: 0.841 - ETA: 15s - loss: 0.6865 - accuracy: 0.838 - ETA: 15s - loss: 0.6816 - accuracy: 0.838 - ETA: 15s - loss: 0.6762 - accuracy: 0.840 - ETA: 15s - loss: 0.6675 - accura

782/782 [==============================] - ETA: 5s - loss: 0.6557 - accuracy: 0.84 - ETA: 4s - loss: 0.6544 - accuracy: 0.84 - ETA: 4s - loss: 0.6540 - accuracy: 0.84 - ETA: 4s - loss: 0.6560 - accuracy: 0.84 - ETA: 4s - loss: 0.6556 - accuracy: 0.84 - ETA: 4s - loss: 0.6551 - accuracy: 0.84 - ETA: 4s - loss: 0.6548 - accuracy: 0.84 - ETA: 4s - loss: 0.6542 - accuracy: 0.84 - ETA: 4s - loss: 0.6527 - accuracy: 0.84 - ETA: 4s - loss: 0.6515 - accuracy: 0.84 - ETA: 4s - loss: 0.6515 - accuracy: 0.84 - ETA: 4s - loss: 0.6529 - accuracy: 0.84 - ETA: 4s - loss: 0.6536 - accuracy: 0.84 - ETA: 4s - loss: 0.6531 - accuracy: 0.84 - ETA: 4s - loss: 0.6517 - accuracy: 0.84 - ETA: 4s - loss: 0.6530 - accuracy: 0.84 - ETA: 4s - loss: 0.6527 - accuracy: 0.84 - ETA: 3s - loss: 0.6539 - accuracy: 0.84 - ETA: 3s - loss: 0.6544 - accuracy: 0.84 - ETA: 3s - loss: 0.6548 - accuracy: 0.84 - ETA: 3s - loss: 0.6567 - accuracy: 0.84 - ETA: 3s - loss: 0.6567 - accuracy: 0.84 - ETA: 3s - loss: 0.6561 - accuracy